In [ ]:
%%capture
!pip install pandas langchain langchain_experimental langchain-groq
!pip install langchain-huggingface
!pip install -qU trl Py7zr auto-gptq optimum
!pip install llama-index-llms-langchain
!pip install faiss-gpu
!pip install ragas
!pip install llama-index-vector-stores-faiss
!pip install llama-index-readers-file
!pip install llama-index-embeddings-huggingface
!pip install llama-index-embeddings-instructor

In [ ]:
import shutil

folder_path = '/kaggle/working/Documents/Images_from_PDF'

# Delete the folder and all its contents
shutil.rmtree(folder_path)


In [ ]:
import os

file_path = '/kaggle/working/Documents/Images_from_PDF'

# Delete the file
if os.path.exists(file_path):
    os.remove(file_path)
    print(f"{file_path} has been deleted.")
else:
    print(f"{file_path} not found.")



In [ ]:
!pip install "unstructured[all-docs]"
!pip install -q unstructured
!apt-get install -y -q poppler-utils
!pip install unstructured-client watermark
!pip install reportlab
!pip install PyPDF2
!pip install pymupdf pdfplumber

In [ ]:
import sys
import os
from time import time
import logging
import torch
import shutil
import faiss
from transformers import AutoTokenizer, AutoModelForCausalLM, GPTQConfig, pipeline, TextStreamer, BitsAndBytesConfig
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain_core.documents import Document
from datasets import load_dataset, Dataset
from pathlib import Path
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.postprocessor import LLMRerank
from IPython.display import Markdown, display
from llama_index.core import Settings
from llama_index.core import (
    SimpleDirectoryReader,
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
)
from llama_index.vector_stores.faiss import FaissVectorStore
from IPython.display import Markdown, display
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import QueryBundle
import pandas as pd
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')
from IPython.display import JSON
import json
from unstructured.partition.html import partition_html
from unstructured.partition.pdf import partition_pdf
from unstructured.staging.base import dict_to_elements, elements_to_json
import unstructured.partition
from unstructured.partition.pdf import partition_pdf
from langchain.schema import Document
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
import nest_asyncio
import glob
from IPython.display import Image, display

nest_asyncio.apply()

In [ ]:
!huggingface-cli login --token '###'

groq_api = '###'
llm = ChatGroq(temperature=0, model="gemma2-9b-it", api_key=groq_api)

# Table Data Extraction

In [ ]:
filename = ""

In [ ]:
elements = partition_pdf(filename=filename,
                         infer_table_structure=True,
                         strategy='hi_res',
           )

element_dict = [el.to_dict() for el in elements]
output = json.dumps(element_dict, indent=2)

unique_types = set()
for item in element_dict:
    unique_types.add(item['type'])
tables = [el for el in elements if el.category == "Table"]

prompt = ChatPromptTemplate.from_messages(
    [("system", "You will throughly explain the numbers/ row components of table \\n\\n{context} with every details. It is important to mention every number. Do not avoid any row. Suppose column head is 'Person', 'Hobby'. And a row is 'A', 'Sleep'. So you will write it as Person A hobby is to sleep." 
     )])

In [ ]:
table_html = tables[0].metadata.text_as_html
from IPython.core.display import HTML
HTML(table_html)

In [ ]:
# Function to save results to PDF with font size 8
def save_results_to_pdf(results, filename="results.pdf"):
    c = canvas.Canvas(filename, pagesize=letter)
    width, height = letter

    cursor_y = height - 40  # Start below the top margin

    for i, result in enumerate(results):
        # Write table result heading
        c.setFont("Helvetica-Bold", 8)
        c.drawString(40, cursor_y, f"Result for Table {i + 1}:")
        cursor_y -= 15  # Move cursor down slightly

        # Write the result content line by line
        c.setFont("Helvetica", 8)
        lines = result.split('\n')  # Handle multi-line content

        for line in lines:
            if cursor_y < 40:  # Create a new page if space runs out
                c.showPage()
                cursor_y = height - 40
                c.setFont("Helvetica", 8)  # Reset font for the new page

            # Break long lines to fit within page width
            while len(line) > 90:  # Assume ~90 chars per line
                part = line[:90]  # Take the first 90 characters
                c.drawString(40, cursor_y, part)
                cursor_y -= 12
                line = line[90:]  # Continue with the remaining part

            c.drawString(40, cursor_y, line)
            cursor_y -= 12  # Move cursor down for the next line

        cursor_y -= 20  # Add space between table results

    c.save()
    print(f"Results saved to {filename}")

# Loop through each table and apply the LLM chain
results = []  # Collect all results

for i, table in enumerate(tables):
    print(f"\nProcessing Table {i + 1}...\n")

    # Safely access HTML content from metadata
    table_html = table.metadata.text_as_html

    if table_html:
        # Create a LangChain Document object with table content
        context = [Document(page_content=table_html)]
        chain = create_stuff_documents_chain(llm, prompt)

        # Invoke the chain for this table
        result = chain.invoke({"context": context})

        # Collect result
        results.append(result)
        print(f"Result for Table {i + 1}:")
        print(result)
    else:
        print(f"HTML content not found for Table {i + 1}.")
        results.append(f"HTML content not found for Table {i + 1}.")

# Save all results to a PDF
#save_results_to_pdf(results, "table_results.pdf")
os.makedirs("/kaggle/working/Documents", exist_ok=True)
save_results_to_pdf(results, "/kaggle/working/Documents/table_results.pdf")

# Image Extraction

In [ ]:
# filename = "/kaggle/input/testing-manual/Transformer Testing Manual.pdf"

In [ ]:
# # Get elements
# path = "/kaggle/working/Documents/Images_from_PDF"
# #filename = "/kaggle/input/testing-manual/Power Cable Testing Manual.pdf"
# raw_pdf_elements = partition_pdf(filename=filename,
#                                  # Unstructured first finds embedded image blocks
#                                  # Only applicable if `strategy=hi_res`
#                                  extract_images_in_pdf=True,
#                                  strategy = "hi_res",
#                                  infer_table_structure=True,
#                                  # Only applicable if `strategy=hi_res`
#                                  extract_image_block_output_dir = path,
#                                  )

# element_dict = [el.to_dict() for el in raw_pdf_elements]

# unique_types = set()

# for item in element_dict:
#     unique_types.add(item['type'])

# print(unique_types)
# images = [el for el in raw_pdf_elements if el.category == "Image"]
# len(images)

In [ ]:
# #Define the path to the folder containing the images
# image_path = "/kaggle/working/Images_from_PDF/figure-18-31.jpg"  # Update the file type as needed

# # Use glob to search for JPG files in the specified folder
# image_files = glob.glob(image_path)

# # Iterate through the list of image files and display each image inline
# for image_file in image_files:
#     display(Image(filename=image_file))

In [ ]:
# import os
# import base64
# from groq import Groq

# # Initialize Groq client
# client = Groq(api_key='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')

# # Function to encode the image
# def encode_image(image_path):
#     with open(image_path, "rb") as image_file:
#         return base64.b64encode(image_file.read()).decode('utf-8')

# # Specific image path
# image_path = '/kaggle/working/Images_from_PDF/figure-18-31.jpg'

# if os.path.isfile(image_path):  # Check if the file exists
#     print(f"\nProcessing Image: {image_path}...\n")

#     # Encode the image
#     base64_image = encode_image(image_path)

#     # Create chat completion request
#     chat_completion = client.chat.completions.create(
#         messages=[
#             {
#                 "role": "user",
#                 "content": [
#                     {"type": "text", "text": "This image will be used in Retrieval Augmented Generation. So explain it as details as you can"},
#                     {
#                         "type": "image_url",
#                         "image_url": {
#                             "url": f"data:image/jpeg;base64,{base64_image}",
#                         },
#                     },
#                 ],
#             }
#         ],
#         model="llama-3.2-90b-vision-preview",
#     )

#     result = chat_completion.choices[0].message.content
#     print(f"Result for Image {image_path}:")
#     print(result)


In [ ]:
# from groq import Groq
# import os
# import base64

In [ ]:
# # Initialize Groq client
# client = Groq(api_key='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')

# # Function to encode the image
# def encode_image(image_path):
#     with open(image_path, "rb") as image_file:
#         return base64.b64encode(image_file.read()).decode('utf-8')

# # Function to save results to PDF with font size 8
# def save_results_to_pdf(results, filename="results.pdf"):
#     c = canvas.Canvas(filename, pagesize=letter)
#     width, height = letter

#     cursor_y = height - 40  # Start below the top margin

#     for i, result in enumerate(results):
#         # Write table result heading
#         c.setFont("Helvetica-Bold", 8)
#         c.drawString(40, cursor_y, f"Result for Image {i + 1}:")
#         cursor_y -= 15  # Move cursor down slightly

#         # Write the result content line by line
#         c.setFont("Helvetica", 8)
#         lines = result.split('\n')  # Handle multi-line content

#         for line in lines:
#             if cursor_y < 40:  # Create a new page if space runs out
#                 c.showPage()
#                 cursor_y = height - 40
#                 c.setFont("Helvetica", 8)  # Reset font for the new page

#             # Break long lines to fit within page width
#             while len(line) > 90:  # Assume ~90 chars per line
#                 part = line[:90]  # Take the first 90 characters
#                 c.drawString(40, cursor_y, part)
#                 cursor_y -= 12
#                 line = line[90:]  # Continue with the remaining part

#             c.drawString(40, cursor_y, line)
#             cursor_y -= 12  # Move cursor down for the next line

#         cursor_y -= 20  # Add space between image results

#     c.save()
#     print(f"Results saved to {filename}")

# # Directory containing images
# images_folder = '/kaggle/working/Documents/Images_from_PDF'
# results = []  # Collect all results

# # Loop through each image in the folder
# for image_name in os.listdir(images_folder):
#     image_path = os.path.join(images_folder, image_name)

#     if os.path.isfile(image_path):  # Check if it's a file
#         print(f"\nProcessing Image: {image_name}...\n")
        
#         # Encode the image
#         base64_image = encode_image(image_path)

#         # Create chat completion request
#         chat_completion = client.chat.completions.create(
#             messages=[
#                 {
#                     "role": "user",
#                     "content": [
#                         {"type": "text", "text": "This image will be used in Retrieval Augmented Generation. So explain it as details as you can"},
#                         {
#                             "type": "image_url",
#                             "image_url": {
#                                 "url": f"data:image/jpeg;base64,{base64_image}",
#                             },
#                         },
#                     ],
#                 }
#             ],
#             model="llama-3.2-90b-vision-preview",
#         )
        
#         result = chat_completion.choices[0].message.content
#         results.append(result)
#         print(f"Result for Image {image_name}:")
#         print(result)

# # Save all results to a PDF
# save_results_to_pdf(results, "/kaggle/working/Documents/image_results.pdf")


# LLM 

In [ ]:
import os
# Step 3: Define source and destination paths
source_path = "/kaggle/input/testing-manual/Transformer Testing Manual.pdf"
destination_dir = "/kaggle/working/Documents"
destination_path = os.path.join(destination_dir, "Transformer Testing Manual.pdf")

# Step 4: Check if destination directory exists, if not, create it
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)

# Step 5: Copy the PDF file to the destination folder
shutil.copy(source_path, destination_path)

# Step 6: Verify if the file has been copied successfully
if os.path.exists(destination_path):
    print("File copied successfully!")
else:
    print("File copy failed.")


# Define the source and destination paths
source_dir = '/kaggle/input/gemma-ft/Gemma_FT'
destination_dir = '/kaggle/working/FineTuned'

# Copy all contents (files and subdirectories) from source to destination
shutil.copytree(source_dir, destination_dir, dirs_exist_ok=True)

In [ ]:
model_ft = "marcsun13/gemma-2-9b-it-GPTQ"

tokenizer_ft = AutoTokenizer.from_pretrained(model_ft)

output_dir_new = "/kaggle/working/FineTuned/finetuned_raft_model"

In [ ]:
quantization_config_loading = GPTQConfig(bits=4, disable_exllama=True)

# disable_exllama=True

model_ft = AutoModelForCausalLM.from_pretrained(output_dir_new,quantization_config=quantization_config_loading, device_map="auto")

pipe_ft = pipeline(
    "text-generation",
    model=model_ft,
    tokenizer=tokenizer_ft,
    max_new_tokens=500,
    do_sample=True,
    temperature=0.01,
    top_p=1,
    top_k=5,
    repetition_penalty=1.1
)

Settings.llm = HuggingFacePipeline(pipeline=pipe_ft)
Settings.chunk_size = 512

In [ ]:
doc = SimpleDirectoryReader("/kaggle/input/shakespeare-searchable").load_data()

# Retrieve 

In [ ]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# dimensions of text-ada-embedding-002
d = 384
faiss_index = faiss.IndexFlatL2(d)

In [ ]:
# loads BAAI/bge-small-en-v1.5
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
Settings.embed_model = embed_model

In [ ]:
vector_store = FaissVectorStore(faiss_index=faiss_index)  # Pass the embed_model

storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    doc, storage_context=storage_context
)

In [ ]:
pd.set_option("display.max_colwidth", None)

def get_retrieved_nodes(
    query_str, vector_top_k=10, reranker_top_n=3, with_reranker=False
):
    query_bundle = QueryBundle(query_str)
    # configure retriever
    retriever = VectorIndexRetriever(
        index=index,
        similarity_top_k=vector_top_k,
    )
    retrieved_nodes = retriever.retrieve(query_bundle)

    if with_reranker:
        # configure reranker
        reranker = LLMRerank(
            choice_batch_size=5,
            top_n=reranker_top_n,
        )
        retrieved_nodes = reranker.postprocess_nodes(
            retrieved_nodes, query_bundle
        )

    return retrieved_nodes


def pretty_print(df):
    return display(HTML(df.to_html().replace("\\n", "<br>")))


def visualize_retrieved_nodes(nodes) -> None:
    result_dicts = []
    for node in nodes:
        result_dict = {"Score": node.score, "Text": node.node.get_text()}
        result_dicts.append(result_dict)

    pretty_print(pd.DataFrame(result_dicts))

In [ ]:
new_nodes = get_retrieved_nodes(
    "",
    vector_top_k=5,
    reranker_top_n=3,
    with_reranker=True,
)

In [ ]:
visualize_retrieved_nodes(new_nodes)

In [ ]:
query_engine = index.as_query_engine(
    similarity_top_k=4,
    node_postprocessors=[
        LLMRerank(
            choice_batch_size=5,
            top_n=2,
        )
    ],
    response_mode="compact",
)

In [ ]:
response = query_engine.query(
    "",
)
print(response)
answer = response.response        # The actual answer

In [ ]:
#context, question, answer= extract_context_question_answer(answer)

In [ ]:
#print(context)
#print(question)
#print(answer)

# Evaluation

In [ ]:
#!pip install datasets==2.14.5 cohere==4.27 chromadb langchain sentence-transformers ragas

In [ ]:
def extract_context_question_answer(data):
    # Step 1: Extract context
    context_end_keyword = "Query:"
    context = data.split(context_end_keyword)[0].strip()

    # Step 2: Extract question and answer
    question_answer = data.split(context_end_keyword)[1].strip()

    # Separate question and answer
    answer_start_keyword = "Answer:"
    question = question_answer.split(answer_start_keyword)[0].strip()
    answer = question_answer.split(answer_start_keyword)[1].strip()

    return context, question, answer

In [ ]:
import time
questions = []

answers = []
contexts = []

question_counter = 0

# Inference
for question in questions:
    start_time = time.time()
    response = query_engine.query(question)

    answer_RAG = response.response        # The actual answer

    # Extract context, question, and answer
    context, question, answer = extract_context_question_answer(answer_RAG)

    answers.append(answer)
    #print(answers)
    contexts.append([context])
    #print(contexts)
    # Increment the counter after processing each question
    question_counter += 1
    end_time = time.time()
    time_taken = end_time - start_time
    # Print or log the current count
    print(f"Question {question_counter} done. Time taken: {time_taken:.2f} seconds")


In [ ]:
print(answers[0:2])

In [ ]:
print(answers[0:5])
#answers[4] = "This document doesn't contain the answer"
#print(answers[4])

In [ ]:
data_5 = {
    "question": questions[4:5],
    "answer": answers[4:5],
    "contexts": contexts[4:5],
    "reference": ground_truths[4:5]
}

# Convert dict to dataset
dataset_5 = Dataset.from_dict(data_5)
print(dataset_5)

In [ ]:
ground_truths = [ ]

data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "reference": ground_truths
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)
print(dataset)


# To dict
data_1 = {
    "question": questions[0:1],
    "answer": answers[0:1],
    "contexts": contexts[0:1],
    "reference": ground_truths[0:1]
}

# Convert dict to dataset
dataset_1 = Dataset.from_dict(data_1)
print(dataset_1)

data_2 = {
    "question": questions[1:2],
    "answer": answers[1:2],
    "contexts": contexts[1:2],
    "reference": ground_truths[1:2]
}

# Convert dict to dataset
dataset_2 = Dataset.from_dict(data_2)
print(dataset_2)

data_3 = {
    "question": questions[2:3],
    "answer": answers[2:3],
    "contexts": contexts[2:3],
    "reference": ground_truths[2:3]
}

# Convert dict to dataset
dataset_3 = Dataset.from_dict(data_3)
print(dataset_3)

data_4 = {
    "question": questions[3:4],
    "answer": answers[3:4],
    "contexts": contexts[3:4],
    "reference": ground_truths[3:4]
}

# Convert dict to dataset
dataset_4 = Dataset.from_dict(data_4)
print(dataset_4)

data_5 = {
    "question": questions[4:5],
    "answer": answers[4:5],
    "contexts": contexts[4:5],
    "reference": ground_truths[4:5]
}

# Convert dict to dataset
dataset_5 = Dataset.from_dict(data_5)
print(dataset_5)

# RAGas Evaluation

In [ ]:
import pandas as pd
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)


In [ ]:
embedding = HuggingFaceEmbeddings()

In [ ]:
llm_RAG = ChatGroq(temperature=0, model="llama-3.3-70b-versatile", api_key=groq_api)

In [ ]:
df_1 = pd.DataFrame()

result_1 = evaluate(
    dataset = dataset_1,
    llm=llm_RAG,
    embeddings=embedding,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall
    ],
)

df_1 = result_1.to_pandas()

In [ ]:
df_1

In [ ]:
df_2 = pd.DataFrame()

result_2 = evaluate(
    dataset = dataset_2,
    llm=llm_RAG,
    embeddings=embedding,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall
    ],
)

df_2 = result_2.to_pandas()
df_2

In [ ]:
df_3 = pd.DataFrame()

result_3 = evaluate(
    dataset = dataset_3,
    llm=llm_RAG,
    embeddings=embedding,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall
    ],
)

df_3 = result_3.to_pandas()
df_3

In [ ]:
df_4 = pd.DataFrame()

result_4 = evaluate(
    dataset = dataset_4,
    llm=llm_RAG,
    embeddings=embedding,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall
    ],
)

df_4 = result_4.to_pandas()
df_4

In [ ]:
df_5 = pd.DataFrame()

result_5 = evaluate(
    dataset = dataset_5,
    llm=llm_RAG,
    embeddings=embedding,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall
    ],
)

df_5 = result_5.to_pandas()
df_5

In [ ]:
import pandas as pd

# Combine the DataFrames
df_combined = pd.concat([df_1, df_2, df_3, df_4, df_5], ignore_index=True)

# Select the last 4 columns
last_4_columns = df_combined.iloc[:, -4:]

# Calculate the mean for each of the last 4 columns
averages = last_4_columns.mean()

# Print the averages with the column headers
for column, avg in averages.items():
    print(f"Average of {column}: {avg}")


# DeepEval

In [ ]:
!pip install -q -q llama-index
!pip install -U -q deepeval

In [ ]:
!deepeval login --confident-api-key "####"

In [ ]:
groq_llm = ChatGroq(temperature=0, model_name="gemma2-9b-it", api_key=groq_api)

In [ ]:
from deepeval import evaluate
from deepeval.metrics import (
    AnswerRelevancyMetric,
    ContextualRelevancyMetric,
    FaithfulnessMetric,
)
from deepeval.test_case import LLMTestCase
from langchain_groq import ChatGroq
from deepeval.models.base_model import DeepEvalBaseLLM

# Create custom DeepEval model using ChatGroq
class GroqModel(DeepEvalBaseLLM):
    def __init__(self, model):
        self.model = model

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        return chat_model.invoke(prompt).content

    async def a_generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        res = await chat_model.ainvoke(prompt)
        return res.content

    def get_model_name(self):
        return "gemma2-9b-it"

# Create test cases from your data
test_cases = [
    LLMTestCase(
        input=question,
        actual_output=answer,
        expected_output=truth,
        retrieval_context=context
    ) for question, answer, truth, context in zip(questions[0:2], answers[0:2], ground_truths[0:2], contexts[0:2])
]

# Initialize all metrics
metrics = [
    AnswerRelevancyMetric(
        model=GroqModel(groq_llm),
        threshold=0.5,
        #evaluation_params=["input", "context"]
    ),
    ContextualRelevancyMetric(
        model=GroqModel(groq_llm),
        threshold=0.6,
        #evaluation_params=["input", "retrieval_context"]
    ),
    FaithfulnessMetric(
        model=GroqModel(groq_llm),
        threshold=0.7,
        #evaluation_params=["input", "context"]
    ),
]

# Run evaluation
if __name__ == "__main__":
    evaluate(
        test_cases,
        metrics=metrics,
        max_concurrent=2,
        #display_progress=True
    )

In [ ]:
from deepeval import evaluate
from deepeval.metrics import (
    AnswerRelevancyMetric,
    ContextualRelevancyMetric,
    FaithfulnessMetric,
)
from deepeval.test_case import LLMTestCase
from langchain_groq import ChatGroq
from deepeval.models.base_model import DeepEvalBaseLLM

# Create custom DeepEval model using ChatGroq
class GroqModel(DeepEvalBaseLLM):
    def __init__(self, model):
        self.model = model

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        return chat_model.invoke(prompt).content

    async def a_generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        res = await chat_model.ainvoke(prompt)
        return res.content

    def get_model_name(self):
        return "gemma2-9b-it"

# Create test cases from your data
test_cases = [
    LLMTestCase(
        input=question,
        actual_output=answer,
        expected_output=truth,
        retrieval_context=context
    ) for question, answer, truth, context in zip(questions[2:4], answers[2:4], ground_truths[2:4], contexts[2:4])
]

# Initialize all metrics
metrics = [
    AnswerRelevancyMetric(
        model=GroqModel(groq_llm),
        threshold=0.5,
        #evaluation_params=["input", "context"]
    ),
    ContextualRelevancyMetric(
        model=GroqModel(groq_llm),
        threshold=0.6,
        #evaluation_params=["input", "retrieval_context"]
    ),
    FaithfulnessMetric(
        model=GroqModel(groq_llm),
        threshold=0.7,
        #evaluation_params=["input", "context"]
    ),
]

# Run evaluation
if __name__ == "__main__":
    evaluate(
        test_cases,
        metrics=metrics,
        max_concurrent=2,
        #display_progress=True
    )

In [ ]:
from deepeval import evaluate
from deepeval.metrics import (
    AnswerRelevancyMetric,
    ContextualRelevancyMetric,
    FaithfulnessMetric,
)
from deepeval.test_case import LLMTestCase
from langchain_groq import ChatGroq
from deepeval.models.base_model import DeepEvalBaseLLM

# Create custom DeepEval model using ChatGroq
class GroqModel(DeepEvalBaseLLM):
    def __init__(self, model):
        self.model = model

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        return chat_model.invoke(prompt).content

    async def a_generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        res = await chat_model.ainvoke(prompt)
        return res.content

    def get_model_name(self):
        return "gemma2-9b-it"

# Create test cases from your data
test_cases = [
    LLMTestCase(
        input=question,
        actual_output=answer,
        expected_output=truth,
        retrieval_context=context
    ) for question, answer, truth, context in zip(questions[4:5], answers[4:5], ground_truths[4:5], contexts[4:5])
]

# Initialize all metrics
metrics = [
    AnswerRelevancyMetric(
        model=GroqModel(groq_llm),
        threshold=0.5,
        #evaluation_params=["input", "context"]
    ),
    ContextualRelevancyMetric(
        model=GroqModel(groq_llm),
        threshold=0.6,
        #evaluation_params=["input", "retrieval_context"]
    ),
    FaithfulnessMetric(
        model=GroqModel(groq_llm),
        threshold=0.7,
        #evaluation_params=["input", "context"]
    ),
]

# Run evaluation
if __name__ == "__main__":
    evaluate(
        test_cases,
        metrics=metrics,
        max_concurrent=2,
        #display_progress=True
    )